In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torchsummary import summary
import time
import os


# 定义卷积神经网络模型
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(7 * 7 * 64, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        return x


# 设置随机种子以保证结果可重复
torch.manual_seed(42)

# 加载MNIST数据集并进行预处理
train_dataset = MNIST(root='./data', train=True, download=True, transform=ToTensor())
test_dataset = MNIST(root='./data', train=False, download=True, transform=ToTensor())

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 创建模型实例并打印模型结构
model = CNN()
summary(model, (1, 28, 28))

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
num_epochs = 10
best_acc = 0.0
best_epoch = 0
checkpoints = './checkpoints'

if not os.path.exists(checkpoints):
    os.makedirs(checkpoints)

for epoch in range(num_epochs):
    # 设置计时器，计算每个epoch的用时
    start_time = time.time()
    model.train()  # 保证每一个batch都能进入model.train()的模式
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    # 记录验证集上准确率最高的模型
    best_model_path = checkpoints + "/" + 'best_model' + '.pth'
    if accuracy >= best_acc:
        best_acc = accuracy
        best_epoch = epoch
        torch.save(model, best_model_path)
    end_time = time.time()
    print(
        f"Epoch [{epoch + 1}/{num_epochs}], Test Accuracy: {accuracy:.2f}%, Time: {end_time - start_time:.4f}s, Best Accuracy: {best_acc:.2f}% at epoch {best_epoch + 1}")
# 保存最后的模型
torch.save(model, checkpoints + '/last.pt')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
              ReLU-2           [-1, 32, 28, 28]               0
         MaxPool2d-3           [-1, 32, 14, 14]               0
            Conv2d-4           [-1, 64, 14, 14]          18,496
              ReLU-5           [-1, 64, 14, 14]               0
         MaxPool2d-6             [-1, 64, 7, 7]               0
            Linear-7                  [-1, 128]         401,536
              ReLU-8                  [-1, 128]               0
            Linear-9                   [-1, 10]           1,290
Total params: 421,642
Trainable params: 421,642
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.65
Params size (MB): 1.61
Estimated Total Size (MB): 2.26
-------------------------------------------

In [27]:
from PIL import Image
import torch


# 预测类
class Pred:
    def __init__(self):
        self.labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
        model_path = 'checkpoints/best_model.pth'
        # 加载模型
        self.model = torch.load(model_path)
        self.device = torch.device('gpu' if torch.cuda.is_available() else 'cpu')
        self.model = self.model.to(self.device)

    # 预测
    def predict(self, img_path):
        transform = transforms.Compose([
            transforms.ToTensor()
        ])
        img = Image.open(img_path)
        img = img.convert('RGB')
        img = transform(img)
        img.resize_(1, 3, 28, 28)
        img = img.view(1, 3, 28, 28).to(self.device)
        output = self.model(img)
        output = torch.softmax(output, dim=1)
        # 每个预测值的概率
        probability = output.cpu().detach().numpy()[0]
        # 找出最大概率值的索引
        output = torch.argmax(output, dim=1)
        index = output.cpu().numpy()[0]
        # 预测结果
        pred = self.labels[index]
        print(pred, probability[index])
        return pred


if __name__ == '__main__':
    img_path = 'mnist/1/1.jpg'
    pred = Pred()
    res = pred.predict(img_path)
    image = Image.open(img_path)
    image.show()
    print(res)

AttributeError: Can't get attribute 'CNN' on <module '__main__'>

In [7]:
import torch
import torchvision
import os
from torchvision import transforms
from torchvision.datasets import mnist
import torch.utils.data as data
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import itertools

train_data = mnist.MNIST('mnist', train=True, transform=transforms.ToTensor(), download=True)
test_data = mnist.MNIST('mnist', train=False, transform=transforms.ToTensor(), download=True)
train_loader = data.DataLoader(dataset=train_data, batch_size=1, shuffle=True)
test_loader = data.DataLoader(dataset=test_data, batch_size=1, shuffle=True)

In [8]:
def show():
    plt.figure(figsize=(16, 9))
    for i, item in enumerate(itertools.islice(train_loader, 2, 102)):
        plt.subplot(10, 10, i + 1)
        img, label = item
        img = img[0].cpu().numpy()
        array = (img.reshape((28, 28)) * 255).astype(np.uint8)
        img = Image.fromarray(array, 'L')
        label = label.cpu().numpy()[0]
        plt.imshow(img, cmap=plt.get_cmap('gray'))
    plt.show()

In [ ]:
import torch
import torch.nn as nn
from torchvision.transforms import ToTensor
from PIL import Image


# 定义卷积神经网络模型
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(7 * 7 * 64, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        return x


# 设置随机种子以保证结果可重复
torch.manual_seed(42)

# 加载MNIST数据集并进行预处理
train_dataset = MNIST(root='./data', train=True, download=True, transform=ToTensor())
test_dataset = MNIST(root='./data', train=False, download=True, transform=ToTensor())

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 创建模型实例并打印模型结构
model = CNN()
summary(model, (1, 28, 28))

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
num_epochs = 10
best_acc = 0.0
best_epoch = 0
checkpoints = './checkpoints'

if not os.path.exists(checkpoints):
    os.makedirs(checkpoints)

for epoch in range(num_epochs):
    # 设置计时器，计算每个epoch的用时
    start_time = time.time()
    model.train()  # 保证每一个batch都能进入model.train()的模式
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    # 记录验证集上准确率最高的模型
    best_model_path = checkpoints + "/" + 'best_model' + '.pth'
    if accuracy >= best_acc:
        best_acc = accuracy
        best_epoch = epoch
        torch.save(model, best_model_path)
    end_time = time.time()
    print(
        f"Epoch [{epoch + 1}/{num_epochs}], Test Accuracy: {accuracy:.2f}%, Time: {end_time - start_time:.4f}s, Best Accuracy: {best_acc:.2f}% at epoch {best_epoch + 1}")
# 保存最后的模型
torch.save(model, checkpoints + '/last.pt')

# 加载训练好的模型
model = CNN()
model.load_state_dict(torch.load('checkpoints/best_model.pth'))
model.eval()

# 加载手写数字图片并进行预处理
image = Image.open('handwritten_digit.png').convert('L')  # 读取手写数字图片并转为灰度图像
image = ToTensor()(image)  # 转换为Tensor格式
image = image.unsqueeze(0)  # 添加batch维度

# 使用模型进行预测
with torch.no_grad():
    prediction = model(image)
    _, predicted_label = torch.max(prediction, 1)

print("Predicted Label:", predicted_label.item())

In [25]:
# excel
import openpyxl
import os

# 打开Excel文件
if not os.path.exists('data.xlsx'):
    workbook = openpyxl.Workbook()
else:
    workbook = openpyxl.load_workbook('data.xlsx')

# 选择要操作的工作表
if 'Sheet1' in workbook.sheetnames:
    worksheet = workbook['Sheet1']
else:
    worksheet = workbook.create_sheet('Sheet1')

# 打开文本文件
with open('ChatGPT.txt', 'r', encoding='UTF-8') as file:
    # 逐行读取文本文件内容
    lines = file.readlines()
    i = 1
    # 逐行写入Excel表格
    for _, line in enumerate(lines):
        if line == '\n':  # 如果是空行，则跳过
            continue
        # 获取要写入的单元格位置
        cell = worksheet.cell(row=i + 1, column=3)
        # 写入数据
        cell.value = line.strip()  # 去除行末尾的换行符
        i += 1

with open('NewBing.txt', 'r', encoding='UTF-8') as file:
    # 逐行读取文本文件内容
    lines = file.readlines()
    i = 1
    # 逐行写入Excel表格
    for _, line in enumerate(lines):
        if line == '\n':  # 如果是空行，则跳过
            continue
        # 获取要写入的单元格位置
        cell = worksheet.cell(row=i + 1, column=4)
        # 写入数据
        cell.value = line.strip()  # 去除行末尾的换行符
        i += 1

# 保存Excel文件
workbook.save('data.xlsx')

In [4]:
# txt
try:
    with open("source.txt", "r", encoding='utf-8') as source:
        f = open("target.txt", "w", encoding='utf-8')
        lines = source.readlines()
        for line in lines:
            if line == '\n':
                continue
            else:
                f.write(line)
except IOError:
    print("Error: 没有找到文件或读取文件失败")
else:
    print("内容写入文件成功")

内容写入文件成功


In [24]:
s = 'hello'
print(eval("s"))  # eval()去除字符串的引号
try:
    print(eval(s))  # s去除引号后，变成了变量名，所以会报错
except NameError:
    print("NameError: name 'hello' is not defined")

hello
NameError: name 'hello' is not defined


In [28]:
# 模式匹配
score = input("请输入成绩：")
match score:
    case "A":
        print("满分")
    case "B":
        print("优秀")
    case "C":
        print("良好")
    case "D":
        print("中等")
    case "E":
        print("及格")
    case _:  # _表示匹配所有其他情况
        print("不及格")

满分


In [5]:
nums = [88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 0, 1]
for i in range(0, len(nums)):
    if nums[i] // 10 != 0:
        nums[i] += 1900
    else:
        nums[i] += 2000
print(nums)

[1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 2000, 2001]


In [6]:
nums = [88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 0, 1]
for index, value in enumerate(nums):
    if value // 10 != 0:
        nums[index] += 1900
    else:
        nums[index] += 2000
print(nums)

[1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 2000, 2001]


In [4]:
info = []
for _ in range(0, 5):
    tmp = input("请输入商品的编号和商品的名称，每次只能输入一个商品：")
    if tmp != "":
        info.append(tmp)
print(info)
cart = []
while True:
    num = input("请输入商品的编号：")
    flag = False
    if num == "q":
        break
    for item in info:
        if num == item[0:4]:
            cart.append(item)
            flag = True
            print("已添加到购物车！")
    if not flag:
        print("您输入的商品编号不存在，请重新输入！")
print("您的购物车中有以下商品：")
cart.reverse()
for item in cart:
    print(item)

['1001平板', '1002手机', '1003耳机', '1004耳机', '1005帽子']
已添加到购物车！
已添加到购物车！
已添加到购物车！
您输入的商品编号不存在，请重新输入！
您的购物车中有以下商品：
1005帽子
1002手机
1001平板


In [6]:
dict_ticket = {
    'G1569': ['北京', '上海', '2019-10-10 08:00', '2019-10-10 10:00', 1200],
    'G1570': ['上海', '北京', '2019-10-10 14:00', '2019-10-10 16:00', 1200],
    'G1566': ['广州', '北京', '2019-10-10 12:00', '2019-10-10 20:00', 2000]
}
print("欢迎使用火车票查询系统！")
print("-" * 30)
users = input("请输出乘车人信息：")
print("您的乘车人信息为：", users)
car = input("请输入要购买的车次：")
if car in dict_ticket.keys():
    print("您选择的车次为：", car)
    print("该车次的详细信息为：", dict_ticket[car])
    print("您的乘车人信息为：", users)
    print("您的车票信息为：", dict_ticket[car])
    print("您的车票已经购买成功，请尽快取票！")
else:
    print("您输入的车次不存在，请重新输入！")
print("-" * 30)

欢迎使用火车票查询系统！
------------------------------
您的乘车人信息为： 吕锦辉
您选择的车次为： G1566
该车次的详细信息为： ['广州', '北京', '2019-10-10 12:00', '2019-10-10 20:00', 2000]
您的乘车人信息为： 吕锦辉
您的车票信息为： ['广州', '北京', '2019-10-10 12:00', '2019-10-10 20:00', 2000]
您的车票已经购买成功，请尽快取票！
------------------------------
